In [19]:
import polars as pl
from excel_manager import get_df, get_count_df, COL_NAME_TOTAL_COUNT

In [20]:
COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY = "flight_with_delay"
COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN = "flight_with_delay_gte_15min"
COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN = "flight_with_delay_41_42"

In [21]:
df = get_df()

In [22]:
df = df.with_columns(pl.col("CODE_DR").cast(pl.Int32).alias("CODE_DR"))

In [23]:
total_df = get_count_df()

In [24]:
delayed_flights_count_df = df.select(
    pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY)
)

delayed_15min_df = df.filter((pl.col("Retard en min") >= 15))
delayed_15min_count_df = delayed_15min_df.select(
    pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN)
)

In [25]:
delayed_count_df = delayed_15min_df.filter((pl.col("CODE_DR").is_in({41, 42}))).select(
    pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN)
)


joined_df = (
    total_df.join(delayed_count_df, how="cross")
    .join(delayed_flights_count_df, how="cross")
    .join(delayed_15min_count_df, how="cross")
    .collect()
)


joined_df

total_count,flight_with_delay_41_42,flight_with_delay,flight_with_delay_gte_15min
u32,u32,u32,u32
17017,194,365,294


In [33]:
count_flight = joined_df.select(pl.col(COL_NAME_TOTAL_COUNT)).item()
count_flight_with_delay = joined_df.select(
    pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY)
).item()
count_flight_with_delay_gte_15min = joined_df.select(
    pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN)
).item()
count_flight_with_delay_gte_15min_41_42 = joined_df.select(
    pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN)
).item()

In [ ]:
# first graph

per_delayed_flights_15min__not_with_41_42 = 1 - (
    count_flight_with_delay_gte_15min_41_42 / count_flight
)

# second graph

## first op (15 min)
per_delayed_flights_not_with_15min = 1 - (
    count_flight_with_delay_gte_15min / count_flight
)


## second op

per_flights_not_delayed = 1 - (count_flight_with_delay / count_flight)


# third graph

17017